In [35]:
import numpy as np
from sklearn import preprocessing
import nltk
import csv
import pandas as pd

import math
import networkx as nx
import community

import random
from sklearn.svm import SVC
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle

import nltk
import csv
from bs4 import BeautifulSoup

from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn import cross_validation
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)
import xgboost as xgb

C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ModuleNotFoundError: No module named 'xgboost'

In [2]:
file = "Period1.csv"
file2 = "Period2.csv"
file3 = "TestData.csv"

def readfile(filename):
    with open(filename, "r") as f:
        reader = csv.reader(f)
        data_set = list(reader)
        
        data_set = [[element[0].split(" ") for element in data_set],[element[1].split(" ") for element in data_set],[element[2].split(" ") for element in data_set]]
        del data_set[0][0]
        del data_set[1][0]
        del data_set[2][0]
        return data_set

In [3]:
data =readfile(file)
data2 = readfile(file2)
testdata = readfile(file3)

In [4]:
def create_features(data):
    features = pd.DataFrame([(data[1][i],data[2][i])for i in range(len(data[0]))])
    features.columns = ['From', 'To']
                    
    year = [i[0] for i in data] 

    features['From'] = features['From'].apply(lambda x:x[0])
    features['To'] = features['To'].apply(lambda x:x[0])
    
    return features

def find_node(node_set1, node_set2):
    node_set = pd.concat([node_set1, node_set2], ignore_index=True)
    node_set_unique = node_set.loc[node_set.duplicated() == False]
    node_set_unique = node_set_unique.reset_index(drop = True)
    return node_set_unique



In [5]:
features = create_features(data)
features2 = create_features(data2)
test = create_features(testdata)

data1_unique_node = find_node(features['From'],features['To'])
data2_unique_node = find_node(features2['From'],features2['To'])
all_unique_node = find_node(data1_unique_node, data2_unique_node)

#implement networkx
data1_edge = [(features['From'][i],features['To'][i])for i in range(len(data[0]))]
data2_edge = [(features2['From'][i],features2['To'][i])for i in range(len(data2[0]))]
all_edge = data1_edge + data2_edge

print('period1 ', 'node:', len(data1_unique_node.values), 'edge:', len(data1_edge))
print('period2 ', 'node:', len(data2_unique_node.values), 'edge:', len(data2_edge))
print('period_all ', 'node:', len(all_unique_node.values), 'edge:', len(all_edge))
print('period_test ', 'node:', len(all_unique_node.values), 'edge:', len(all_edge))
print(data1_edge)


period1  node: 17028 edge: 154836
period2  node: 15565 edge: 98353
period_all  node: 23237 edge: 253189
period_test  node: 23237 edge: 253189
[('9909133', '9811241'), ('9702110', '9410212'), ('9510041', '9210121'), ('9703036', '9611090'), ('9610162', '9609159'), ('9905085', '9803171'), ('9303003', '9202046'), ('9604189', '9601152'), ('9601175', '9507050'), ('9801084', '9705128'), ('9809104', '9601116'), ('9703138', '9603081'), ('9609181', '9604168'), ('9701137', '9606165'), ('9607051', '9401006'), ('9608169', '9409009'), ('9709113', '9608015'), ('9505073', '9207053'), ('9811217', '9802202'), ('9805083', '9510182'), ('9908147', '9808101'), ('9704118', '9208007'), ('9803189', '9602052'), ('9812028', '9609070'), ('9707014', '9610043'), ('9902184', '9801052'), ('9802168', '9709193'), ('9607210', '9505051'), ('9910090', '9902055'), ('9509147', '9509142'), ('9512026', '9304087'), ('9511230', '9511204'), ('9803117', '9612115'), ('9501042', '9310042'), ('9903214', '9203018'), ('9903029', '9812

In [6]:
def create_graph(nodes, edges):
    G = nx.Graph()
    
#     linked_nodes = [(data['From'][i],data['To'][i])for i in range(len(data))]
    G.add_nodes_from(nodes.values)
    
    G.add_edges_from(edges)
    print('node:', G.number_of_nodes(), 'edge:', G.number_of_edges())
    
    return G

def create_digraph(nodes, edges):
    G = nx.DiGraph()
    
#     linked_nodes = [(data['From'][i],data['To'][i])for i in range(len(data))]
    G.add_nodes_from(nodes.values)

    G.add_edges_from(edges)
    print('node:', G.number_of_nodes(), 'edge:', G.number_of_edges())
    
    return G

def labeling(edges,complement):
    label = []
    for edge in edges:
        label.append(1)
        
    for edge in list(complement.edges()):
        label.append(0)
#     for i in range(features.shape[0]):
#         a = features['From'][i]
#         b = features['To'][i]
#         if(a in features2.values or b in features2.values):
#             label.append(1)
#         else:
#             label.append(0)
    return label

# def draw_graph()

In [7]:
graph = create_graph(all_unique_node, data1_edge)
graph2 = create_digraph(all_unique_node, data1_edge)
nx.is_directed(graph2)

node: 23237 edge: 154698
node: 23237 edge: 154836


True

In [8]:
full_graph = create_graph(all_unique_node, all_edge)
full_graph2 = create_digraph(all_unique_node, all_edge)

node: 23237 edge: 252854
node: 23237 edge: 253189


In [9]:
def common_neighbors(features, G):
    nb_common_neighbors = []
    for edge_id, edge in enumerate(features):
        a = edge[0]
        b = edge[1]
        nb_common_neighbors.append(len(sorted(nx.common_neighbors(G, a, b)))) # ajoute le nombre de voisins communs
    return nb_common_neighbors

def Jaccard_coef(features, G):
    J = []
    for edge_id, edge in enumerate(features):
        a = edge[0]
        b = edge[1]
        pred = nx.jaccard_coefficient(G,[(a,b)])
        for u,v,p in pred:
            J.append(p)
    return J

def betweenness_diff(features, G):
    btw = nx.betweenness_centrality(G, 50)
    btw_diff = []
    for edge_id, edge in enumerate(features):
        a = edge[0]
        b = edge[1]
        btw_diff.append(btw[b] - btw[a])
    return btw_diff

def in_link_diff(features, G2):
    diff = []
    for edge_id, edge in enumerate(features):
        a = edge[0]
        b = edge[1]
        diff.append(len(G2.in_edges(b)) - len(G2.in_edges(a)))
    return diff

def is_same_cluster(partition, features):
    same_cluster = []
    for edge_id, edge in enumerate(features):
        a = edge[0]
        b = edge[1]
        if(partition[a] == partition[b]):
            same_cluster.append(1)
        else:
            same_cluster.append(0)
    return same_cluster

def adamic_adar(node1,node2, G):
    adam = []
    adamic_score = 0
    a = [n for n in G.neighbors(node1)]
    b = [n for n in G.neighbors(node2)]
    intersection = list(set(a) & set(b))
    
    if len(intersection) == 0:
        return 0
    else:
        for v in intersection:
            adamic_score += 1 / math.log(len([nv for nv in G.neighbors(v)]))
        return adamic_score
    
def clustering_coefficient(G, node):
    node_degree = G.degree[node]
    node_triangle = nx.triangles(G, node)
    if node_degree - 1 <= 0:
        return 0
    else:
        return (2 * node_triangle) / (node_degree * (node_degree - 1)) 
    
def preferential_attachment(G, node1, node2): 
    a = len([n for n in G.neighbors(node1)])
    b = len([n for n in G.neighbors(node2)])
    return {'pa_mul': a * b, 'pa_add': a + b}

In [10]:
#generate model

in1_notin2 = data1_unique_node.append(data2_unique_node).drop_duplicates(keep=False)
data_shuffle = random.Random(2).sample(list(in1_notin2), 450)
sub_graph = graph.subgraph(data_shuffle)
# graph_complement = nx.complement(graph)
sub_graph_complement = nx.complement(sub_graph)

#tagging label
train_label = labeling(data2_edge,sub_graph_complement)

train_data_edge = data2_edge + list(sub_graph_complement.edges())
train_data = pd.DataFrame(data={'edge': train_data_edge, 'label': train_label})
train_data = shuffle(train_data, random_state=32).reset_index(drop=True)
train_data.head()



,edge,label
0,"(3273, 9901003)",1
1,"(6022, 6169)",0
2,"(9504016, 9703225)",0
3,"(9061, 9805170)",1
4,"(102045, 9901128)",1


In [11]:
no_common_neighbors = common_neighbors(train_data['edge'].values, full_graph)
train_data['No_common_neighbors'] = no_common_neighbors

Jaccard = Jaccard_coef(train_data['edge'].values, full_graph)
train_data['Jaccard_coef'] = Jaccard

btw_diff = betweenness_diff(train_data['edge'].values, full_graph)
train_data['Betweenness_diff'] = btw_diff

# diff = in_link_diff(train_data['edge'].values, graph2)
# features['In_link_diff'] = diff

partition = community.best_partition(full_graph)
same_cluster_train = is_same_cluster(partition, train_data['edge'].values)
train_data['Is_same_cluster'] = same_cluster_train

cc_mul, cc_add, pa_mul, pa_add,adamic = [],[], [], [],[]
source_id, target_id = [],[]
for edge_id, edge in enumerate(train_data['edge'].values):
    source_id.append(edge[0])
    target_id.append(edge[1])
    
    adamic.append(adamic_adar(edge[0], edge[1], full_graph))
    source_cc = clustering_coefficient(full_graph, edge[0])
    target_cc = clustering_coefficient(full_graph, edge[1])
    cc_mul.append(source_cc * target_cc)
    cc_add.append(source_cc + target_cc)
    pa = preferential_attachment(full_graph, edge[0], edge[1])
    pa_mul.append(pa['pa_mul'])
    pa_add.append(pa['pa_add'])
    
    if edge_id % 10000 == 0:
        print(edge_id, len(train_data))
        
train_data['adamic'] = pd.Series(adamic, index=train_data.index)
train_data['cc_mul'] = pd.Series(cc_mul, index=train_data.index)
train_data['cc_add'] = pd.Series(cc_add, index=train_data.index)
train_data['pa_mul'] = pd.Series(pa_mul, index=train_data.index)
train_data['pa_add'] = pd.Series(pa_add, index=train_data.index)
    

0 199365
10000 199365
20000 199365
30000 199365
40000 199365
50000 199365
60000 199365
70000 199365
80000 199365
90000 199365
100000 199365
110000 199365
120000 199365
130000 199365
140000 199365
150000 199365
160000 199365
170000 199365
180000 199365
190000 199365


In [13]:

#save train data
train_data_bye = train_data.drop('edge', axis=1)
train_data_bye.to_csv("train_data.csv", index=False)


PermissionError: [Errno 13] Permission denied: 'train_data.csv'

In [14]:


# result_list.iloc[0:10000].to_csv('result.csv', sep=',')
# features.to_csv('graph_features_training.csv', sep=',')
train_data = pd.read_csv("train_data.csv")

In [15]:
feature = zip(train_data['No_common_neighbors'], train_data['Jaccard_coef'],train_data['Betweenness_diff'], train_data['adamic'], train_data['cc_mul'], train_data['cc_add'],
              train_data['pa_mul'], train_data['pa_add'],train_data['Is_same_cluster'])
feature = [[No_common_neighbors, Jaccard_coef,Betweenness_diff, adamic, cc_mul, cc_add, pa_mul,pa_add,Is_same_cluster]
           for No_common_neighbors, Jaccard_coef,Betweenness_diff, adamic, cc_mul, cc_add, pa_mul, pa_add, Is_same_cluster in feature]


In [16]:
#svm
# SVM

# clf = ExtraTreesClassifier(max_features=None, min_samples_leaf=20, n_estimators = 500, n_jobs=3)

# clf.fit(X_train, y_train)
# pred = clf.predict(X_test)

svm = SVC()
svm.fit(feature, train_data['label'].tolist())
# svm.predict([[0, 0, 0, 0, 0]])


In [26]:
# Decision tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(feature, train_data['label'].tolist())

In [ ]:
# Random forest
rf = RandomForestClassifier(random_state=0)
rf = dt.fit(feature, train_data['label'].tolist())

In [ ]:
# KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn = knn.fit(feature, train_data['label'].tolist())

In [ ]:
# extra trees classifier

clf = ExtraTreesClassifier(max_features=None,min_samples_leaf=10,n_estimators=500,n_jobs=3)
cv = cross_validation.cross_val_score(clf, X_train, y_train, cv=5)


In [ ]:
#xg boost classifier
# 1st tuning

gbm = xgb.XGBClassifier(max_depth=6, n_estimators=500, learning_rate=0.01)
cv = cross_validation.cross_val_score(gbm, X_train, y_train, cv=5)
print(np.mean(cv))



In [ ]:
# 2nd tuning
gbm = xgb.XGBClassifier(max_depth=4, n_estimators=500, learning_rate=0.05)
gbm.fit(X_train, y_train)
pred = gbm.predict(X_test)


In [ ]:
#grid search
from sklearn.grid_search import GridSearchCV
parameters = {'n_estimators':[500,1000],
        'learning_rate': [0.05, 0.01, 0.001]}

clf = GridSearchCV( xgb.XGBClassifier(max_depth=4), parameters, n_jobs=4, cv=5, verbose = 10)
clf.fit(X_train, y_train)

In [19]:

# data1_edge = [(features['From'][i],features['To'][i])for i in range(len(data[0]))]
# data2_edge = [(features2['From'][i],features2['To'][i])for i in range(len(data2[0]))]
# all_edge = data1_edge + data2_edge

# step 3 link predict
# check test data is in network
test_data_unique_node = find_node(test['From'],test['To'])
test_data_network = pd.concat([all_unique_node, test_data_unique_node], ignore_index=True)
test_data_network = test_data_network.loc[test_data_network.duplicated() == True]
test_data_network = test_data_network.reset_index(drop=True)


# Add test data node & edge in network, implement networkx
test_edge = [(test['From'][i],test['To'][i])for i in range(len(test))]
network_test = full_graph
network_test.add_nodes_from(test_data_unique_node.values)
network_test.add_edges_from(test_edge)

print('period_test ', 'node:', network_test.number_of_nodes(), 'edge:', network_test.number_of_edges())

period_test  node: 31491 edge: 262850


In [25]:
#testing

no_common_neighbors_test = common_neighbors(test_edge, network_test)
# test_data_network['No_common_neighbors'] = no_common_neighbors_test

Jaccard_test = Jaccard_coef(test_edge, network_test)
# test_data_network['Jaccard_coef'] = Jaccard_test

btw_diff_test = betweenness_diff(test_edge, network_test)
# test_data_network['Betweenness_diff'] = btw_diff_test

# diff = in_link_diff(train_data['edge'].values, graph2)
# features['In_link_diff'] = diff

partition = community.best_partition(network_test)
same_cluster_train_test = is_same_cluster(partition, test_edge)
# train_data['Is_same_cluster'] = same_cluster_train

cc_mul_test, cc_add_test, pa_mul_test, pa_add_test,adamic_test = [],[], [], [],[]
source_id, target_id = [],[]
for edge_id, edge in enumerate(test_edge):
    source_id.append(edge[0])
    target_id.append(edge[1])
    
    adamic_test.append(adamic_adar(edge[0], edge[1], network_test))
    source_cc = clustering_coefficient(network_test, edge[0])
    target_cc = clustering_coefficient(network_test, edge[1])
    cc_mul_test.append(source_cc * target_cc)
    cc_add_test.append(source_cc + target_cc)
    pa = preferential_attachment(network_test, edge[0], edge[1])
    pa_mul_test.append(pa['pa_mul'])
    pa_add_test.append(pa['pa_add'])
    
#     if edge_id % 10000 == 0:
#         print(edge_id, len(train_data))
        
# train_data['adamic'] = pd.Series(adamic, index=train_data.index)
# train_data['cc_mul'] = pd.Series(cc_mul, index=train_data.index)
# train_data['cc_add'] = pd.Series(cc_add, index=train_data.index)
# train_data['pa_mul'] = pd.Series(pa_mul, index=train_data.index)
# train_data['pa_add'] = pd.Series(pa_add, index=train_data.index)
    


In [33]:

test_feature = [[n, j,b,clus, a, cc_mul, cc_add, pa_mul, pa_add] for n, j,b,clus, a, cc_mul, cc_add, pa_mul,pa_add in 
                zip(no_common_neighbors_test, Jaccard_test,btw_diff_test,same_cluster_train_test, adamic_test, cc_mul_test, cc_add_test,
                    pa_mul_test,pa_add_test)]

predict = dt.predict(test_feature)
print(predict)

[0 0 0 ... 1 0 0]


In [34]:
# output predict
row = [i for i in range(1, 10001)]
label = predict

data = {'target id': row, 'label': label}
predict = pd.DataFrame(data=data, columns=['target id', 'label'])
predict.to_csv("dt_nei_cc_pamul.csv", index=False)


#submission
# predict = list(predict) 
# predict = zip(range(len(predict)), predict)
# with open("result_pred_3.csv","w") as pred1:
#     csv_out = csv.writer(pred1)
#     csv_out.writerow(["target id", "label"])
#     for row in predict:
#         csv_out.writerow(row)

# with open("result_pred_3.csv", "r") as f:
#     reader = csv.reader(f)
#     data_set = list(reader)
# #     data_set = [[element[0].split(" ") for element in data_set],[element[1].split(" ") for element in data_set]]  
    
# result = pd.DataFrame(data_set[1][i] for i in range(1,len(data_set[0])))
# result.index = np.arange(1, len(result) + 1)
# result.index.name = 'target id'
# result.columns = ['label']
# result.iloc[0:10000].to_csv('result_pred_3.csv', sep=',')


In [41]:
# features2.to_csv('graph_features_testing.csv', sep=',')

In [33]:
# label = labeling(features, features2)
# features['label'] = label

In [44]:
# features.to_csv('graph_features_training_with_label.csv', sep=',')

In [46]:
#prediction

train = pd.read_csv('graph_features_training.csv')
del train['Unnamed: 0']


In [47]:
#training
X_train = train.as_matrix()

y_train = label

#testing
test = pd.read_csv('graph_features_testing.csv')
del test['Unnamed: 0']

X_test = test.as_matrix()


0.93311633282


98353


In [75]:
#TestData
features_test = create_features(testdata)


In [77]:
#testing
graph_testdata = create_graph(features_test)
graph_testdata_2 = create_digraph(features_test)

no_common_neighbors_testdata = common_neighbors(features_test, graph_testdata)
features_test['No_common_neighbors'] = no_common_neighbors_testdata

Jaccard_testdata = Jaccard_coef(features_test, graph_testdata)
features_test['Jaccard_coef'] = Jaccard_testdata

btw_diff_testdata = betweenness_diff(features_test, graph_testdata)
features_test['Betweenness_diff'] = btw_diff_testdata


diff3 = in_link_diff(features_test, graph_testdata_2)
features_test['In_link_diff'] = diff3

partition3 = community.best_partition(graph_testdata)
same_cluster_testdata = is_same_cluster(partition3, features_test)
features_test['Is_same_cluster'] = same_cluster_testdata

# result_list.iloc[0:10000].to_csv('result.csv', sep=',')
features_test.to_csv('graph_features_testing_data.csv', sep=',')

In [78]:
#testing
testing = pd.read_csv('graph_features_testing_data.csv')
del testing['Unnamed: 0']

X_test2 = testing.as_matrix()

In [79]:
pred_test = clf.predict(X_test2)

In [80]:
pred_test = gbm.predict(X_test2)

In [81]:
#submission
predictions = list(pred_test) 
predictions = zip(range(len(pred_test)), predictions)
with open("result_pred_2.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(["ID", "category"])
    for row in predictions:
        csv_out.writerow(row)

In [82]:


with open("result_pred_2.csv", "r") as f:
    reader = csv.reader(f)
    data_set = list(reader)
    data_set = [[element[0].split(" ") for element in data_set],[element[1].split(" ") for element in data_set]]  
    
result = pd.DataFrame(data_set[1][i] for i in range(1,len(data_set[0])))
result.index = np.arange(1, len(result) + 1)
result.index.name = 'target id'
result.columns = ['label']
result.iloc[0:10000].to_csv('result_pred_2.csv', sep=',')


In [23]:
print(test_data_network)

0        208165
1        111208
2        202140
3        112113
4        108010
5        203020
6        110022
7        111044
8        109191
9        105097
10       203003
11       111228
12       204066
13       102127
14       110028
15       112180
16       204158
17       110113
18       106149
19       105255
20         5143
21       104041
22       202188
23       203235
24       112126
25         1077
26       107242
27       203208
28       109069
29       203268
         ...   
2609    9912018
2610    9707133
2611    9303017
2612    9811089
2613     104037
2614    9908151
2615      12249
2616    9901033
2617    9711043
2618    9701192
2619    9508154
2620    9506104
2621    9504155
2622     107123
2623    9410010
2624     103153
2625    9610018
2626       9014
2627     107147
2628    9305125
2629    9409110
2630    9701028
2631    9808010
2632     109194
2633    9706201
2634    9811022
2635     110109
2636    9608111
2637       5275
2638    9706139
Length: 2639, dtype: obj